In [0]:
from pyspark.sql.functions import regexp_extract

# Monitoriza el directorio logs y lee las líneas
accessLines = spark.readStream.text("dbfs:/FileStore/logs")

# Parsea las líneas según el formato del log
contentSizeExp = r'\s(\d+)$'
statusExp = r'\s(\d{3})\s'
generalExp = r'\"(\S+)\s(\S+)\s*(\S*)\"'
timeExp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
hostExp = r'(^\S+\.[\S+\.]+\S+)\s'

logsDF = accessLines.select(regexp_extract('value', hostExp, 1).alias('host'),
                         regexp_extract('value', timeExp, 1).alias('timestamp'),
                         regexp_extract('value', generalExp, 1).alias('method'),
                         regexp_extract('value', generalExp, 2).alias('endpoint'),
                         regexp_extract('value', generalExp, 3).alias('protocol'),
                         regexp_extract('value', statusExp, 1).cast('integer').alias('status'),
                         regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size'))

# Contabiliza el número de accesos por status code
statusCountsDF = logsDF.groupBy(logsDF.status).count()

# Arranca el procesamiento en streaming, volcando datos en la consola.
# Envía la salida al log de la consola del driver, no al notebook web. 
# Es útil en entornos tipo terminal, pero Databricks no muestra por defecto esa salida en las celdas del notebook.

# query = ( statusCountsDF.writeStream.outputMode("complete").format("console").queryName("counts").start() )
# query.awaitTermination()

# display(dataframe) es un comando específico de Databricks que:
#   - Ejecuta una consulta de streaming en modo “append” o “update”.
#   - Muestra los resultados visualmente dentro del notebook, en una tabla o gráfico interactivo.
#   - Y lo hace internamente con su propio displayStreamQuery.

display(statusCountsDF)
